In [17]:
import nltk
from nltk.stem.porter import PorterStemmer
import string
import re
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
num_regex = re.compile('^[+-]?[0-9]+\.?[0-9]*$')
from sklearn.cross_validation import train_test_split

/home/saddam/ML/venv3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
stemmer = LancasterStemmer()
def stem_tokens(tokens,stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

################### Stop Word Remove ##################

stop_words = set(stopwords.words('english'))

#punctuation list
s = list(string.punctuation)

def tokenize(string):
    tokens = nltk.word_tokenize(string)
    for index, token in enumerate(tokens):
        if token == '@' and (index+1) < len(tokens):
            tokens[index+1] = '@' + re.sub('[0-9]+.*', '', tokens[index+1])
            tokens.pop(index)
    #remove english stop word from tokens
    filtered_sentence = [w for w in tokens if not w in stop_words]
    
    #remove punctuation 
    tt = [i for i in filtered_sentence if not i in s]
    
    #Apply stemming 
    stems_word = stem_tokens(tt,stemmer)
    
    return stems_word

In [3]:
data = pd.read_excel('training_set_rel3.xls')
data.head()
df = data[data['essay_set']==2]
x = df['essay']
Y = df['domain1_score']

In [4]:
tfidf = TfidfVectorizer(tokenizer=tokenize,min_df=1,max_features=1000).fit(x)
X = tfidf.transform(x)

In [5]:
sing = X.toarray()
sing.shape

(1800, 1000)

In [7]:
U, s, V = np.linalg.svd(sing, full_matrices=False)
m = 0
for i in range(1000):
    j = float(s[i])
    if j >= 1.3:
          m =m+1
print(m)

238


In [11]:
reduce = len(s)-m
#print reduce
ua = U.shape[1]-reduce
u = U[:,:ua]
#print(u.shape)
v = V.shape[0]-reduce
v = V[:v]
####################################################
from numpy import diag
S= diag(s[:m])
S.shape

(238, 238)

In [12]:
u = np.matrix(u)
s = np.matrix(S)
v = np.matrix(v)

In [13]:
svd_matrix = u*s*v

In [14]:
svd_matrix.shape

(1800, 1000)

In [15]:
from scipy import sparse
full_svd = sparse.csc_matrix(svd_matrix)

In [18]:
train_x,test_x,train_y,test_y= train_test_split(full_svd,Y,random_state=0,test_size=0.1)

In [22]:
#SVM

In [23]:
from sklearn import svm
model = svm.SVC(C=0.7,gamma=1)
model.fit(train_x,train_y)
model.score(test_x,test_y)

0.65555555555555556

In [24]:
from sklearn.linear_model import SGDRegressor
sgmodel = SGDRegressor(alpha=0.0001,max_iter=1000)
sgmodel.fit(train_x,train_y)
sgmodel.score(test_x,test_y)

0.47631655908206516

In [26]:
from sklearn.metrics import mean_absolute_error,mean_squared_log_error,mean_squared_error,median_absolute_error,r2_score


In [27]:
print('SGDregressor mean_squared_log_error :%f' % mean_squared_log_error(test_y,sgmodel.predict(test_x)))
print( 'SGDregressor mean_absolute_error :%f'%mean_absolute_error(test_y,sgmodel.predict(test_x)))
print( 'SGDregressor median_absolute_error :%f' % median_absolute_error(test_y,sgmodel.predict(test_x)))
print('SGDregressor mean_squared_error :%f' % mean_squared_error(test_y,sgmodel.predict(test_x)))
print( 'SGDregressor r2_score :%f' % r2_score(test_y,sgmodel.predict(test_x)))

SGDregressor mean_squared_log_error :0.023001
SGDregressor mean_absolute_error :0.438112
SGDregressor median_absolute_error :0.365052
SGDregressor mean_squared_error :0.325847
SGDregressor r2_score :0.476317


In [28]:
pd.DataFrame({'Actual':test_y.tolist(),'SVM':model.predict(test_x).tolist(),'SGRegra':sgmodel.predict(test_x).tolist()})

,Actual,SGRegra,SVM
0,3.0,3.939418,3.0
1,5.0,4.581016,4.0
2,3.0,3.469318,3.0
3,4.0,3.602556,4.0
4,4.0,4.547684,4.0
5,4.0,3.795801,4.0
6,4.0,3.000466,3.0
7,3.0,3.721250,4.0
8,4.0,3.380844,4.0
9,3.0,3.497375,4.0


In [29]:
svm_model = model.predict(test_x).tolist()
sgd_model = sgmodel.predict(test_x).tolist()

In [40]:
final_ac = []
for i in range(180):
    s_samble = float((float(svm_model[i]*3.0)+float(sgd_model[i]*0.5))/3.5)
    final_ac.append(s_samble)


In [41]:
pd.DataFrame({'Actual':test_y.tolist(),'B_samble':final_ac,'SVM':model.predict(test_x).tolist(),'SGRegra':sgmodel.predict(test_x).tolist()})

,Actual,B_samble,SGRegra,SVM
0,3.0,3.134203,3.939418,3.0
1,5.0,4.083002,4.581016,4.0
2,3.0,3.067045,3.469318,3.0
3,4.0,3.943222,3.602556,4.0
4,4.0,4.078241,4.547684,4.0
5,4.0,3.970829,3.795801,4.0
6,4.0,3.000067,3.000466,3.0
7,3.0,3.960179,3.721250,4.0
8,4.0,3.911549,3.380844,4.0
9,3.0,3.928196,3.497375,4.0


In [42]:
print('SGDregressor mean_squared_log_error :%f' % mean_squared_log_error(test_y,final_ac))
print( 'SGDregressor mean_absolute_error :%f'%mean_absolute_error(test_y,final_ac))
print( 'SGDregressor median_absolute_error :%f' % median_absolute_error(test_y,final_ac))
print('SGDregressor mean_squared_error :%f' % mean_squared_error(test_y,final_ac))
print( 'SGDregressor r2_score :%f' % r2_score(test_y,final_ac))

SGDregressor mean_squared_log_error :0.029664
SGDregressor mean_absolute_error :0.385260
SGDregressor median_absolute_error :0.064529
SGDregressor mean_squared_error :0.402676
SGDregressor r2_score :0.352842
